# OpenAI APIの前準備

In [ ]:
# パッケージのインストール
!pip install -U openai

In [ ]:
import os
from google.colab import userdata

# 環境変数の準備 (左端の鍵アイコンでOPENAI_API_KEYを設定)
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [ ]:
from openai import OpenAI

# クライアントの準備
client = OpenAI()

# テキスト生成の実行

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "developer",
        "content": "ひとことで応答を返すアシスタントです。"
    },
    {
        "role": "user",
        "content": "日本一高い山は？",
    }
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",  # モデルID
    input=messages  # メッセージリスト
)
print(response.output_text)

In [ ]:
# トークン数の確認
print("入力トークン数:", response.usage.input_tokens)
print("出力トークン数:", response.usage.output_tokens)
print("合計トークン数:", response.usage.total_tokens)

# テキスト生成の主なタスク
## 文章生成

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "user",
        "content": "サイバーパンクな赤ずきんの物語を書いてください。",
    }
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages
)
print(response.output_text)

## 質問応答

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "developer",
        "content": "人工知能について教えて下さい。",
    }
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages
)
print(response.output_text)

## 要約

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "user",
        "content": """以下の文章を短い1文で要約してください。
「OpenAI」は、営利法人「OpenAI LP」とその親会社である非営利法人「OpenAI Inc.」からなる人工知能の研究所です。2015年末にサム・アルトマン、イーロン・マスクらによってサンフランシスコで設立されました。人類全体に利益をもたらす形で友好的なAIを普及・発展させることを目標に掲げています。""",
    }
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages
)
print(response.output_text)

## 翻訳

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "user",
        "content": """以下の日本語を英語に翻訳してください。
吾輩は猫である"""
    }
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages
)
print(response.output_text)

## コーディング

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "user",
        "content": "'Hello World!'と表示するPythonコードを作成してください"
    }
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages
)
print(response.output_text)

# ストリーミング

In [ ]:
# メッセージリストの準備
messages = [
    {
        "role": "developer",
        "content": "長々と応答を返すアシスタントです。"
    },
    {
        "role": "user",
        "content": "日本一高い山は？",
    }
]

In [ ]:
# 推論の実行
stream = client.responses.create(
    model="gpt-5",
    input=messages,
    stream=True,  # ストリーミングの有効化
)

# ストリーミングの出力
for event in stream:
    print(event)

In [ ]:
# 推論の実行
stream = client.responses.create(
    model="gpt-4o",
    input=messages,
    stream=True,  # ストリーミングの有効化
)

# ストリーミングの出力 (テキスト差分のみ)
for event in stream:
    if event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)

# 会話状態管理

In [ ]:
# メッセージリストの準備
messages = [
    {"role": "user", "content": "日本一高い山は？"},
    {"role": "assistant", "content": "富士山です。"},
    {"role": "user", "content": "その山は何県にある？"},
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages,
)
print(response.output_text)

In [ ]:
# 最初の推論の実行
response = client.responses.create(
    model="gpt-5",
    input=[{"role": "user", "content": "日本一高い山は？"}],
)
print(response.output_text)

In [ ]:
# previous_response_id で会話状態を引き継いで推論
second_response = client.responses.create(
    model="gpt-5",
    previous_response_id=response.id,
    input=[{"role": "user", "content": "その山は何県にある？"}],
)
print(second_response.output_text)

# 構造化出力

In [ ]:
# メッセージリストの準備
messages = [
    {"role": "developer", "content": "イベント情報を抽出します。"},
    {"role": "user", "content": "アリスとボブは金曜日に科学フェアに行く予定です。"}
]

In [ ]:
# JSONスキーマの準備
schema = {
    "format": {
        "type": "json_schema",
        "name": "calendar_event",
        "schema": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string"
                },
                "date": {
                    "type": "string"
                },
                "participants": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                },
            },
            "required": ["name", "date", "participants"],
            "additionalProperties": False
        },
        "strict": True
    }
}

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",
    input=messages,
    text=schema
)
print(response.output_text)

# プロンプトキャッシュ

In [ ]:
# 参照テキストの準備
with open("akazukin_all.txt", "r", encoding="utf-8") as file:
    target_text = file.read()
print(target_text)

In [ ]:
messages=[
    {
        "role": "developer",
        "content": [{"type": "input_text", "text": target_text}]
    },
    {
        "role": "user",
        "content": [{"type": "input_text", "text": "質問: 主人公の名前は？"}]
    },
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",  # モデルID
    input=messages,  # メッセージリスト
    prompt_cache_key="my-cache-key-123", # プロンプトキャッシュキー
)
print(response.output_text)

# トークン数の確認
print("入力トークン数:", response.usage.input_tokens)
print("  キャッシュトークン数:", response.usage.input_tokens_details.cached_tokens)
print("出力トークン数:", response.usage.output_tokens)
print("合計トークン数:", response.usage.total_tokens)

In [ ]:
messages=[
    {
        "role": "developer",
        "content": [{"type": "input_text", "text": target_text+target_text}]
    },
    {
        "role": "user",
        "content": [{"type": "input_text", "text": "質問: 主人公は何をしている？"}]
    },
]

In [ ]:
# 推論の実行
response = client.responses.create(
    model="gpt-5",  # モデルID
    input=messages,  # メッセージリスト
    prompt_cache_key="my-cache-key-123", # プロンプトキャッシュキー
)
print(response.output_text)

# トークン数の確認
print("入力トークン数:", response.usage.input_tokens)
print("  キャッシュトークン数:", response.usage.input_tokens_details.cached_tokens)
print("出力トークン数:", response.usage.output_tokens)
print("合計トークン数:", response.usage.total_tokens)